In [1]:
import torch
import transformers
from fastcoref import FCoref
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import AutoModelForCausalLM
from transformers import MistralConfig, BitsAndBytesConfig

In [2]:
quantization = '4bit' # Valid values: None, '8bit', '4bit'
rag_adapter_path = './fine_tuning/fine_tuned_models'
torch.cuda.memory._record_memory_history(max_entries=100000)

# Load Mistral into memory

In [3]:
base_model = "mistralai/Mistral-7B-Instruct-v0.2"
# Choose quantization type
if quantization == '8bit':
    bnb_config = BitsAndBytesConfig(
        load_in_8bit=True,
    )
    model = AutoModelForCausalLM.from_pretrained(base_model, config=MistralConfig, quantization_config=bnb_config, device_map='cuda')
    print('Model loaded with 8-bit quantization')
elif quantization == '4bit':
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_use_double_quant=False,
    )
    model = AutoModelForCausalLM.from_pretrained(base_model, config=MistralConfig, quantization_config=bnb_config, device_map='cuda')
    print('Model loaded with 4-bit quantization')
else:
    model = AutoModelForCausalLM.from_pretrained(base_model, config=MistralConfig, device_map='cuda')
    print('Model loaded with no quantization')

/home/matlab/miniconda3/envs/llm_student_msc/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Model loaded with 4-bit quantization


### Load adapters on top of Mistral

In [4]:
model.load_adapter(rag_adapter_path, adapter_name='rag_adapter')
model.set_adapter('rag_adapter')

# Load FCoref components

In [5]:
coref_model = FCoref()

05/19/2024 18:28:53 - INFO - 	 missing_keys: []
05/19/2024 18:28:53 - INFO - 	 unexpected_keys: []
05/19/2024 18:28:53 - INFO - 	 mismatched_keys: []
05/19/2024 18:28:53 - INFO - 	 error_msgs: []
05/19/2024 18:28:53 - INFO - 	 Model Parameters: 90.5M, Transformer: 82.1M, Coref head: 8.4M


# Load Nomic embedder

In [6]:
embedding_model = HuggingFaceEmbeddings(
    model_name="nomic-ai/nomic-embed-text-v1",
    model_kwargs={
        'device': 'cuda',
        'trust_remote_code': True
        }
    )

05/19/2024 18:28:53 - INFO - 	 Load pretrained SentenceTransformer: nomic-ai/nomic-embed-text-v1
/home/matlab/miniconda3/envs/llm_student_msc/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
05/19/2024 18:28:57 - WARNING - 	 <All keys matched successfully>


In [7]:
torch.cuda.memory._dump_snapshot('vram_usage_4bit_everything.pickle')
torch.cuda.memory._record_memory_history(enabled=None)